# Traitement et stockage des données issues du scraping

Vous avez scrapé les données du site de livres et les avez stockées dans un fichier. 

L'objectif de ce notebook est de créer une base de données pour y stocker ces données.

In [41]:
import sqlite3
import pandas as pd

Lire les données du fichier sauvegardé en utilisant pandas.

In [42]:
# Lire les données du fichier que vous venez d'enregistrer
df_books = pd.read_csv("books_infos.csv")

## 1. Prétraitement des données

On souhaite créer la table _book_ contenant les attributs suivants : 
- id : INT, PK,
- title : TEXT,
- price : DECIMAL
- availability : BOOLEAN
- rating : INT [0:5]

Vérifier les types des colonnes du dataframe.

In [43]:
# Vérification des types de données
print(df_books.dtypes)

title           object
price           object
rating          object
availability    object
dtype: object


Dans les cellules qui suivent, des méthodes de traitement de données sont suggérées pour donner un aperçu de ce qu'il est possible de faire avec pandas.

**Il est tout à fait possible de faire autrement.**

Utiliser la méthode pandas [_astype_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.astype.html) pour convertir la colonne de titre en chaîne de caractère.

In [44]:
# Conversion de title en chaîne de caractères
df_books["title"] =df_books["title"].astype(str)

# Vérification du type de la colonne title
print(df_books["title"].dtype)

object


Pour convertir la colonne de prix en nombre décimal, il est nécessaire d'utiliser une étape intermédiaire pour retirer le caractère "£".

Il est possible par exemple d'utiliser l'attribut [.str](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html) de la série "price".

In [45]:
# Convertir la colonne price en type décimal
df_books["price"] = df_books["price"].astype(str).str.replace("£", "").astype(float)

# Vérification du type de la colonne price
print(df_books["price"].dtype)

float64


Convertir la colonne `availability` en boolen (True/False).

Quelles sont les valeurs possibles pour la colonne availability ?

In [ ]:
# Valeurs possibles de la colonne availability
df_books["availability"].unique()

# Convertir la colonne availability en booléen (True/False)
# Ici, on considère que 'In stock' signifie que le livre est disponible
df_books['availability_bool'] = df_books['availability'].apply(lambda x: 'In stock' in x)
#lambda: fonction anonyme qui prend une valeur x
#teste si la chaîne 'In stock' est présente dans la valeur x. Résultat : True ou False.

# Vérification des valeurs uniques de la colonne availability_bool
print(df_books['availability_bool'].unique())


[ True]


Créer une fonction qui prend en entrée la valeur de `availability` et qui renvoie True ou False en fonction de la valeur d'entrée.

In [49]:
# Fonction pour convertir la valeur de availability en booléen
def convert_availability(value : str) -> bool:
    """Convert the availability value to a boolean.

    Args:
        value (str): The availability status of the book.

    Returns:
        bool: True if the book is available, False otherwise.
    """
    return 'In stock' in value    

Utiliser la méthode [`apply`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html) pour appliquer la fonction à la colonne `availability`.

In [50]:
# Convertir la colonne availability en booléen (True/False)
df_books["availability"] = df_books["availability"].apply(convert_availability)

# Vérification du type de la colonne availability
print(df_books["availability"].dtype)

bool


Convertir la colonne _rating_ en chiffre en utilisant un dictionnaire `rating_map` et la méthode [_map_](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.map.html).

In [51]:
# Dictionnaire associant les notes au format initial et les valeurs numérique
ratings_map ={
    'One': 1,
    'Two': 2,
    'Three': 3,
    'Four': 4,
    'Five': 5
}

df_books["rating"] =df_books["rating"].map(ratings_map)

# Vérification du type de la colonne rating
print(df_books["rating"].dtype)

int64


In [ ]:
# Créer une fonction convert_types qui combine les traitements faits dans les cellules précédentes
def convert_types(df_books: pd.DataFrame) -> pd.DataFrame:
    """Convert the types of the DataFrame columns to appropriate types.

    Args:
        df_books (pd.DataFrame): The DataFrame containing book data.

    Returns:
        pd.DataFrame: The DataFrame with converted types.
    """
 
    # Dictionnaire pour convertir les notes textuelles en chiffres
    ratings_map = {
        'One': 1,
        'Two': 2,
        'Three': 3,
        'Four': 4,
        'Five': 5
    }
    # Fonction pour convertir la disponibilité en booléen
    def convert_availability(value: str) -> bool:
        return 'In stock' in value

    # Convertir availability en booléen
    df_books["availability"] = df_books["availability"].apply(convert_availability)

    # Convertir rating en entier
    df_books["rating"] = df_books["rating"].map(ratings_map)

    # Convertir price en float 
    df_books["price"].dtype ==df_books["price"].astype(str).str.replace("£", "").astype(float)

    return df_books



---
## 2. Insertion des données en base

Dans cette section :
- on créé une BDD sqlite  `book_store.db` (ou on se connecte à la base si elle existe déjà) en utilisant la bibliothèque python sqlite3,
- on insère les données prétraitées dans la BDD

Utiliser le [tutoriel](https://www.ionos.fr/digitalguide/sites-internet/developpement-web/sqlite3-avec-python/) pour l'utilisation de sqlite3.

Utiliser la fonction pandas adaptée qui permet d'insérer un dataframe dans une BDD.

In [71]:
def create_database(df_books: pd.DataFrame, db_name: str = "books.db") -> None:
    """Crée une base de données SQLite et insère les données de livres.

    Args:
        df_books (pd.DataFrame): Le DataFrame contenant les données des livres.
        db_name (str): Le nom du fichier de base de données.
    """
    # Convertir les types de données 
    df_books = convert_types(df_books)

    # Connexion à la base de données SQLite
    conn = sqlite3.connect(db_name)

    # Enregistrement des données dans une table appelée 'books'
    df_books.to_sql("books", conn, if_exists="replace", index=False)

    # Fermeture de la connexion
    conn.close()


Vérifier le nombre de livres présents dans la BDD en utilisant sqlite3 et la requête SQL adaptée.

In [ ]:
#1/ Vérifie si le DataFrame existe dans la session
"df_books" in locals() or "df_books" in globals()

True

In [ ]:
#2/ Vérifie les colonnes
print(df_books.columns.tolist())



['title', 'price', 'rating', 'availability', 'availability_bool']


In [ ]:
# Vérifier les 5 premières lignes
 df_books.head()


,title,price,rating,availability,availability_bool
0,A Light in the Attic,51.77,3,True,True
1,Tipping the Velvet,53.74,1,True,True
2,Soumission,50.10,1,True,True
3,Sharp Objects,47.82,4,True,True
4,Sapiens: A Brief History of Humankind,54.23,5,True,True


In [ ]:
#Enregistrer  df_books dans la base SQLite

import sqlite3

# Connexion à la base de données
conn = sqlite3.connect("books.db")

# Sauvegarde du DataFrame dans une table appelée 'books'
df_books.to_sql("books", conn, if_exists="replace", index=False)

# Fermeture de la connexion
conn.close()

print("Données enregistrées dans la base SQLite avec succès.")


Données enregistrées dans la base SQLite avec succès.


In [ ]:
# Tester la lecture depuis la base
count_books()         # Devrait maintenant afficher le bon nombre
show_first_books()    # Devrait afficher les 5 premiers livres


Nombre de livres dans la base : 1000
                                   title  price  rating  availability  \
0                   A Light in the Attic  51.77       3             1   
1                     Tipping the Velvet  53.74       1             1   
2                             Soumission  50.10       1             1   
3                          Sharp Objects  47.82       4             1   
4  Sapiens: A Brief History of Humankind  54.23       5             1   

   availability_bool  
0                  1  
1                  1  
2                  1  
3                  1  
4                  1  
